# Polynomials

Work-In-Progress: This is an experimental notebook

In [1]:
from polynomials import *

There are only two classes involved here: **Term** and **Poly**.

A **Poly** is a list of **Term**.

**Example**:

$-2 -4x +7x^2 -3x^4$ is a polynomial that consists of four terms,

where each term has a coefficient, order, and the same variable:

$-2x^0$ and $-4x^1$ and $+7x^2$ and $-3x^4$.

## Terms

A ``Term`` consists of:

* **coefficient** (``int`` or ``float``)
* **order** (non-negative ``int``)
* **varname** (``string`` variable name)

The term $-2y^3$ is represented as follows:

In [2]:
t1 = Term(-2, 3, 'y')
print(t1)
t1

-2y^3


Term(-2, 3, y)

Within a Jupyter notebook like this one, the function, ``latex``, will output a string using Latex formatting.

In [3]:
latex(t1)

<IPython.core.display.Math object>

If no variable name is entered, then the default is **'x'**.

The default variable name is not output in the print representation.

In [4]:
t2 = Term(1, 2)
print(t2)
latex(t2)
t2

+1x^2


<IPython.core.display.Math object>

Term(1, 2)

### Term: Equality

The method, ``unpack``, returns the three quantities that make up a Term.  It's used below to create a copy of a Term and then check for equality.

In [5]:
coef, ordr, var = t1.unpack()
t1b = Term(coef, ordr, var)
print(f"t1  = {t1}")
print(f"t1b = {t1b}")
print(f"t1 == t1b ? {t1 == t1b}")

t1  = -2y^3
t1b = -2y^3
t1 == t1b ? True


The method, ``copy``, will create an exact copy of a Term.

In [6]:
t1c = t1.copy()
print(f"t1 == t1c ? {t1 == t1c}")
print(f"t1  object ID: {id(t1)}")
print(f"t1c object ID: {id(t1c)}")

t1 == t1c ? True
t1  object ID: 140163451730064
t1c object ID: 140163452387856


### Term: Power

Terms can be raised to a non-negative integer power.

In [7]:
latex(t1)  # Test with this term

<IPython.core.display.Math object>

In [8]:
t3a = t1**2
latex(t3a)
t3a

<IPython.core.display.Math object>

Term(4, 6, y)

Raising a term to the 0 power yields 1, as expected.

In [9]:
t3b = t1**0
latex(t3b)
t3b

<IPython.core.display.Math object>

Term(1, 0, y)

### Term: Multiplication

Terms can be multiplied by an ``int`` or a ``float``, on the left or the right.

In [10]:
num = 5.1  # Test with this value,
latex(t3a)  # and this term

<IPython.core.display.Math object>

In [11]:
t4 = num * t3a
latex(t4)
t4

<IPython.core.display.Math object>

Term(20.4, 6, y)

In [12]:
num * t3a == t3a * num

True

### Term: Like terms

Like terms have the same variable and the same order.

e.g., $-2x^7$ and $8x^7$ are like terms.
<p>$-2x^7$ and $8x^3$ are **not** like terms.</p>

In [13]:
# Test with these terms

t5 = Term(-2, 7)
t6 = Term( 8, 7)
t7 = Term( 8, 3)

latex(f"t5 = {t5}")
latex(f"t6 = {t6}")
latex(f"t7 = {t7}")

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [14]:
t6.like_term(t5)

True

In [15]:
t6.like_term(t7)

False

### Term: Substitution

Terms are callable using numeric values and even other terms ("variable substitution")

**Example**:

If $t_1 = -2y^3$ and $y=5$,

then $t_1(5) = -2(5^3) = -250$

In [16]:
show_subst(t1, 5)

(-2y^3)(y=5) ==> -250


-250

A Term can be called (evaluated) at another Term.

Effectively, this allows for **variable substitution**.

**Example**:

If $t_1 = -2y^3$ and $t_2 = x^2$,

then $t_1(t_2) = -2(x^2)^3 = -2x^6$

and $t_2(t_1) = (-2y^3)^2 = 4y^6$

In [17]:
latex(f"t1 = {t1}")
latex(f"t2 = {t2}")

latex(f"t1(t2) = t1({t2}) = {t1(t2)}")
latex(f"t2(t1) = t2({t1}) = {t2(t1)}")

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [18]:
show_subst(t1, t2)

(-2y^3)(y=+1x^2) ==> -2x^6


Term(-2, 6)

In [19]:
show_subst(t2, t1)

(+1x^2)(x=-2y^3) ==> +4y^6


Term(4, 6, y)

Evaluating a constant term with a term yields the constant again, but with the variable of the term that was "plugged into" the constant term.

In [20]:
t0x = Term(7, 0, 'x')
t0y = Term(4, 0, 'y')
print(t0x)
print(t0y)

+7
+4


Plug the following terms into the two constant terms above.

In [21]:
t1

Term(-2, 3, y)

In [22]:
t2  # Variable is the default, 'x'

Term(1, 2)

In [23]:
show_subst(t0x, t1)

(+7)(x=-2y^3) ==> +7


Term(7, 0, y)

In [24]:
show_subst(t0x, t2)

(+7)(x=+1x^2) ==> +7


Term(7, 0)

In [25]:
t0y(t1)

Term(4, 0, y)

In [26]:
t0y(t2)

Term(4, 0)

We can check if a term is *constant*, *linear*, or *quadratic*.  If it is, then the coeffient of the term is returned, otherwise ``False`` is returned.

In [27]:
t0x

Term(7, 0)

In [28]:
t0x.is_constant()

7

In [29]:
t0x.is_linear()

False

In [30]:
t0x.is_quadratic()

False

## Polynomials

A polynomial, ``Poly``, can be constructed from a 1D array of integers or floats that represent coefficients, where the index of a coefficient in the array is its order in the polynomial.

Other methods of polynomial creation are described farther below.

In [31]:
coeffs = [-2, -4, 7, 0, -3]

p = Poly(coeffs)
p

Poly([-2, -4, 7, 0, -3])

By default, 'x' is assumed to be the variable in a polynomial.

In [32]:
print(p)

-2 -4x +7x^2 -3x^4


In [33]:
latex(p)

<IPython.core.display.Math object>

A ``Poly`` is callable.

In [34]:
a = 3  # A test value

In [35]:
p(a)

-194

Here's a check of the arithmetic in the call made, above.

In [36]:
print(-2 - 4*a + 7*a**2 - 3*a**4)

-194


In [37]:
show_subst(p, a)

(-2 -4x +7x^2 -3x^4)(x=3) ==> -194


-194

### Evaluate a Term at a Polynomial

In [38]:
t8 = Term(-2, 2, 'y')
latex(t8)  # The Term

<IPython.core.display.Math object>

In [39]:
latex(p)  # The polygon

<IPython.core.display.Math object>

In [43]:
latex(show_subst(t8, p))

(-2y^2)(y=-2 -4x +7x^2 -3x^4) ==> -8 -32x +24x^2 +112x^3 -122x^4 -48x^5 +84x^6 -18x^8


<IPython.core.display.Math object>

### Polygon Structure

Internally, a ``Poly`` is made up of a list of ``Terms`` and a ``varname`` (variable name string).

A polynomial's term list is immutable, so it is a *property*; however, ``varname`` is a method, because it can be used to change the polynomials variable (e.g., 'x' -> 'y')

In [44]:
p.terms

[Term(-2, 0), Term(-4, 1), Term(7, 2), Term(-3, 4)]

In [45]:
p.varname()

'x'

A ``Term`` can be retrieved, by its order, using the polynomial's ``term`` method.

In [46]:
t2 = p.term(2)
t2

Term(7, 2)

Here's an example of a polynomial with *varname*, 'y'.

In [47]:
py = Poly(coeffs, 'y')
py

Poly([-2, -4, 7, 0, -3], 'y')

In [52]:
py.terms

[Term(-2, 0, y), Term(-4, 1, y), Term(7, 2, y), Term(-3, 4, y)]

In [53]:
latex(py)

<IPython.core.display.Math object>

In [54]:
show_subst(py, 3)

(-2 -4y +7y^2 -3y^4)(y=3) ==> -194


-194

## Other Ways to Construct Polynomials

### From a String Representation

A ``Poly`` can also be constructed from a string representation of a polynomial, such as "-2 -4x +7x^2 -3x^4", where...

* there **must** be a space between each term,
* and, except for the first term, every term must begin with a + or - sign, and then a number, unless the number is a 1.

In [55]:
polystr = "-2 -4x +7x^2 -3x^4"

p2 = Poly(polystr)
print(p2)
p2

-2 -4x +7x^2 -3x^4


Poly([-2, -4, 7, 0, -3])

In [56]:
polystry = "7 +3y -y^2 -9y^3"

p2y = Poly(polystry)

In [57]:
p2y.terms

[Term(7, 0, y), Term(3, 1, y), Term(-1, 2, y), Term(-9, 3, y)]

In [58]:
print(p2y)
latex(p2y)

+7 +3y -1y^2 -9y^3


<IPython.core.display.Math object>

### From a List of Pairs (Tuples)

In [59]:
coef_order_pairs_a = [(-2,0), (-4,1), (7, 2), (-3, 4)]

p3a = Poly(coef_order_pairs_a)
print(p3a)
p3a

-2 -4x +7x^2 -3x^4


Poly([-2, -4, 7, 0, -3])

### From a List of Pairs (Lists)

In [60]:
coef_order_pairs_b = [[-2,0], [-4,1], [7, 2], [-3, 4]]

p3b = Poly(coef_order_pairs_b)
print(p3b)
p3b

-2 -4x +7x^2 -3x^4


Poly([-2, -4, 7, 0, -3])

### From a List of Terms

In [61]:
list_of_terms = [Term(-2,0), Term(-4,1), Term(7, 2), Term(-3, 4)]

p4 = Poly(list_of_terms)
print(p4)
p4

-2 -4x +7x^2 -3x^4


Poly([-2, -4, 7, 0, -3])

### From a List of Roots

In [62]:
roots = [1, 2, 3]

p5 = from_roots(roots)
print(p5)
p5

+6 -11x +6x^2 -1x^3


Poly([6, -11, 6, -1])

## Combining Like Terms

Like terms (i.e., same order terms) are automatically combined during various polynomial operations, include the construction of polynomials.

In [63]:
test_terms = [Term(-2,0), Term(-4,1), Term(-1,0), Term(7,2), Term(-3,4), Term(-2,2)]

In [64]:
print(test_terms)
print(combine_like_terms(test_terms))

[Term(-2, 0), Term(-4, 1), Term(-1, 0), Term(7, 2), Term(-3, 4), Term(-2, 2)]
[Term(-3, 0), Term(-4, 1), Term(5, 2), Term(-3, 4)]


In [65]:
polystr2 = "-2 -4x -1 +7x^2 -3x^4 -2x^2"
p2 = Poly(polystr2)
print(p2)
p2

-3 -4x +5x^2 -3x^4


Poly([-3, -4, 5, 0, -3])

In [66]:
polystr3 = "-2 +3x^4 -4x +2 +7x^2 +4x -3x^4 -7x^2"
p3 = Poly(polystr3)
print(p3)
p3

0


Poly([0])

## Testing Some Polynomial Strings

In [67]:
polystr = "-2 -4x^1 +7x^2 -3x^4"  # Added '^1' to linear term

p = Poly(polystr)
print(p)
p.terms

-2 -4x +7x^2 -3x^4


[Term(-2, 0), Term(-4, 1), Term(7, 2), Term(-3, 4)]

In [68]:
polystr = "-2x^0 -4x^1 +7x^2 -3x^4"  # Added 'x^0' to constant term

p = Poly(polystr)
print(p)
p.terms

-2 -4x +7x^2 -3x^4


[Term(-2, 0), Term(-4, 1), Term(7, 2), Term(-3, 4)]

In [69]:
polystr = "-4x -2 -3x^4 +7x^2"  #  Terms not in order 

p = Poly(polystr)
print(p)
p.terms

-2 -4x +7x^2 -3x^4


[Term(-2, 0), Term(-4, 1), Term(7, 2), Term(-3, 4)]

In [70]:
polystr = "2 -x -3 +7x^2 -3x^4"  # No coefficient on linear term, 'x'

p = Poly(polystr)
print(p)
p.terms

-1 -1x +7x^2 -3x^4


[Term(-1, 0), Term(-1, 1), Term(7, 2), Term(-3, 4)]

In [71]:
polystr = "2 -x -3 +7x^2 +x -3x^4"  # linear terms cancel

p = Poly(polystr)
print(p)
p.terms

-1 +7x^2 -3x^4


[Term(-1, 0), Term(7, 2), Term(-3, 4)]

In [72]:
polystr = "-4x^2 -x -3 +7x^2 +x -3x^2"  # Quadratic terms collapse and cancel

p = Poly(polystr)
print(p)
p.terms

-3


[Term(-3, 0)]

## Polynomial Arithmetic

Polynomials can be added, subtracted, multiplied, negated, and raised to a positive integer power.

In [73]:
q1 = Poly('1 +2x')
print(f"q1 = {q1}")

q2 = Poly('1 -2x')
print(f"q2 = {q2}")

q1 = +1 +2x
q2 = +1 -2x


In [74]:
print(q1 + 1)
print(1 + q1)

+2 +2x
+2 +2x


In [75]:
print(q1)
print(-q1)
print(3*q1)

+1 +2x
-1 -2x
+3 +6x


In [76]:
print(1 + q1)
print(q1 + 1)

+2 +2x
+2 +2x


In [77]:
print(1 + -q1)
print(1 - q1)

-2x
-2x


In [78]:
print(q1 - 1)

+2x


In [79]:
print(f"({q1}) + ({q2}) = {q1 + q2}")

(+1 +2x) + (+1 -2x) = +2


In [80]:
print(f"({q1}) - ({q2}) = {q1 - q2}")

(+1 +2x) - (+1 -2x) = +4x


In [81]:
print(f"({q1}) * ({q2}) = {q1 * q2}")

(+1 +2x) * (+1 -2x) = +1 -4x^2


In [82]:
print(f"({q1}) * ({q1}) = {q1 * q1}")

(+1 +2x) * (+1 +2x) = +1 +4x +4x^2


In [83]:
print(f"({q2}) * ({q2}) = {q2 * q2}")

(+1 -2x) * (+1 -2x) = +1 -4x +4x^2


In [84]:
print(f"-({q1}) = {-q1}")

-(+1 +2x) = -1 -2x


In [85]:
print(f"-({q2}) = {-q2}")

-(+1 -2x) = -1 +2x


In [86]:
print(f"({q1})**2 = {q1**2}")

(+1 +2x)**2 = +1 +4x +4x^2


In [87]:
print(f"({q2})**2 = {q2**2}")

(+1 -2x)**2 = +1 -4x +4x^2


## Polynomial Representation

The representation of a polynomial changes between two options, depending the estimated length of the representation.  The representation heuristic tries to produce the shortest representation possible, while still allow it to be one that can be used to recreate the polynomial.

In [88]:
coeffs1 = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7]
p_large_order1 = Poly(coeffs1)
p_large_order1

Poly([(1, 0), (7, 19)])

In [89]:
coeffs2 = [1, 0, 0, 0, 0, 0, 7]
p_large_order2 = Poly(coeffs2)
p_large_order2

Poly([(1, 0), (7, 6)])

In [90]:
coeffs3 = [1, 0, 0, 0, 0, 7]
p_large_order3 = Poly(coeffs3)
p_large_order3

Poly([1, 0, 0, 0, 0, 7])

## Using Floats

Polynomial coefficients can be floats, as well as integers, or a mix of the two.

In [91]:
coeffs = [-2.2, -4, 7.3, -0.0, -3]

p1 = Poly(coeffs)
print(p1)
p1

-2.2 -4x +7.3x^2 -3x^4


Poly([-2.2, -4, 7.3, 0, -3])

In [92]:
p2 = Poly('-2.2 -4x +7.3x^2 -3x^4')
print(p2)
p2

-2.2 -4x +7.3x^2 -3x^4


Poly([-2.2, -4, 7.3, 0, -3])

## Polynomial Derivatives and Antiderivatives

In [93]:
print(p1)
p1_der = p1.derivative()
print(p1_der)

-2.2 -4x +7.3x^2 -3x^4
-4 +14.6x -12x^3


In [94]:
p1_der_antider = p1_der.antiderivative(111)
print(p1_der_antider)

+111 -4.0x +7.3x^2 -3.0x^4


## Scratchwork

In [95]:
r = Poly('-3 +2x +7x^2')
latex(f"r = {r}")

<IPython.core.display.Math object>

In [96]:
s = Poly('5 +4y', 'y')
latex(f"s = {s}")

<IPython.core.display.Math object>

In [98]:
show_subst(s, r)

(+5 +4y)(y=-3 +2x +7x^2) ==> -7 +8x +28x^2


Poly([-7, 8, 28])

In [99]:
r0 = Poly('-3')
show_subst(s, r0)

(+5 +4y)(y=-3) ==> -7


Poly([-7])

In [100]:
r1 = Poly('2x')
show_subst(s, r1)

(+5 +4y)(y=+2x) ==> +5 +8x


Poly([5, 8])

In [101]:
r2 = Poly('7x^2')
show_subst(s, r2)

(+5 +4y)(y=+7x^2) ==> +5 +28x^2


Poly([5, 0, 28])

In [110]:
# show_subst(r, s)  # ValueError: Mult: Variables must be the same, x != y

# NumPy Polynomials

This is a better library for polynomials.

In [103]:
from numpy.polynomial import Polynomial

poly_coeff = [-2, -4, 7, 0, -3]
poly = Polynomial(poly_coeff)
poly

Polynomial([-2., -4.,  7.,  0., -3.], domain=[-1,  1], window=[-1,  1])

In [104]:
poly(3)

-194.0

In [105]:
print(poly)

-2.0 - 4.0·x¹ + 7.0·x² + 0.0·x³ - 3.0·x⁴


In [106]:
poly1 = Polynomial([1, 1])
poly1

Polynomial([1., 1.], domain=[-1,  1], window=[-1,  1])

In [107]:
poly2 = Polynomial([1, -1])
poly2

Polynomial([ 1., -1.], domain=[-1,  1], window=[-1,  1])

In [108]:
poly1 * poly2

Polynomial([ 1.,  0., -1.], domain=[-1.,  1.], window=[-1.,  1.])

In [111]:
poly1(poly2)

Polynomial([ 2., -1.], domain=[-1.,  1.], window=[-1.,  1.])

In [112]:
sn = Polynomial([5, 4])
rn = Polynomial([-3, 2, 7])

In [113]:
sn

Polynomial([5., 4.], domain=[-1,  1], window=[-1,  1])

In [114]:
rn

Polynomial([-3.,  2.,  7.], domain=[-1,  1], window=[-1,  1])

In [115]:
sn(rn)

Polynomial([-7.,  8., 28.], domain=[-1.,  1.], window=[-1.,  1.])

In [116]:
rn(sn)

Polynomial([182., 288., 112.], domain=[-1.,  1.], window=[-1.,  1.])